In [5]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *

# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *

In [6]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

In [7]:
train_emb, train_emb_api, train_fam_ans, train_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/TRAIN_vec.pkl','rb'))
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/DEV_vec.pkl','rb'))
test_emb, test_emb_api,test_fam_ans,test_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/TEST_vec.pkl','rb'))
# print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
# print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
valid_rep_ans = np.expand_dims(valid_rep_ans,axis=-1)
test_rep_ans = np.expand_dims(test_rep_ans,axis=-1)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)
emb_matrix = pickle.load(open('data/tree-rep-profiles-partial/api_emb_matrix.pkl','rb'))

test of sent2vec vector: (424, 213, 768) (424, 213) (424, 44) (424, 213, 1)


In [8]:
def _shuffle(X, X2 ,X3,X4):
#     X3 = np.take(train_fam_ans,[0],axis=-1) #只train第幾個familiy
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], X2[randomize],X3[randomize],X4[randomize])

In [9]:
train_emb, train_emb_api, train_fam_ans, train_rep_ans = _shuffle(train_emb, train_emb_api, train_fam_ans, train_rep_ans)
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = _shuffle(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans)

# test_emb, test_emb_api,test_fam_ans,test_rep_ans  = _shuffle(test_emb,test_emb_api,test_fam_ans,test_rep_ans)

print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)

# print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

train of sent2vec vector: (11141, 213, 768) (11141, 213) (11141, 44) (11141, 213, 1)
valid of sent2vec vector: (437, 213, 768) (437, 213) (437, 44) (437, 213, 1)


In [6]:
# scale = 'no'

# dim-wise scaling
def scaling(trainX,validX,testX,scale='min_max'):
#     if scale == 'min_max':
    max_value = max([np.max(trainX) , np.max(validX),np.max(testX)])
    min_value = min([np.min(trainX),np.min(validX),np.min(testX)])

    trainX = (trainX - min_value) / (max_value - min_value)
    validX = (validX - min_value) / (max_value - min_value )
    testX = (testX - min_value) / (max_value - min_value )
    print(np.max(trainX),np.max(validX))
    return trainX,validX,testX , max_value , min_value
def scaling(trainX,validX,testX,scale='mean_dim',):
#     if scale == 'min_max':
    alls = np.concatenate((trainX,validX,testX),axis=0)
    mean = np.mean(alls,axis=-1)
    mean = np.mean(mean,axis=0)
    mean = np.expand_dims(mean,axis=-1)
    mean = np.repeat(mean,trainX.shape[2],axis=-1)
    mean = np.expand_dims(mean,axis=0)
    mean_train = np.repeat(mean,trainX.shape[0],axis=0)
    mean_valid = np.repeat(mean,validX.shape[0],axis=0)
    mean_test = np.repeat(mean,testX.shape[0],axis=0)
    std = np.std(alls,axis=-1)
    std = np.std(std,axis=0)
    std = np.expand_dims(std,axis=-1)
    std = np.repeat(std,validX.shape[2],axis=-1)
    std_train = np.repeat(std,trainX.shape[0],axis=0)
    std_valid = np.repeat(std,validX.shape[0],axis=0)
    std_test = np.repeat(std,testX.shape[0],axis=0)
#     min_value = min([np.min(trainX),np.min(validX),np.min(testX)])

    trainX = (trainX - mean) / (std + 1e-10)
    validX = (validX - mean) / (std + 1e-10)
    testX = (testX - mean) / (std + 1e-10)
#     print(np.max(trainX),np.max(validX))
    return trainX,validX,testX , mean , std

# train_emb,valid_emb,test_emb , max_value,min_value = scaling(train_emb,valid_emb,test_emb)   
# print(valid_emb.shape)

In [7]:
train_emb[3]

array([[-0.33855778, -0.32893312, -0.72717506, ...,  0.91245544,
         1.18298519, -0.31610405],
       [-0.48375982, -0.76325315, -1.20088184, ...,  0.87430948,
         1.2489109 , -0.08723355],
       [-0.22453758, -0.52563572, -1.29923677, ...,  0.74041533,
         1.04407859, -0.49547631],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [8]:
train_emb.shape[2]

768

In [9]:
# kk = np.mean(train_emb,axis=-1)
# kk = np.mean(kk,axis=0)
# kk = np.expand_dims(kk,axis=0)
# kk = np.repeat(kk,100,axis=0)
# kk = np.expand_dims(kk,axis=-1)
# kk = np.repeat(kk,768,axis=-1)
# kk.shape

In [10]:
# kkk = (emb_matrix - kk)/kk


In [11]:
# kk = np.std(emb_matrix,axis=-1)
# kk = np.expand_dims(kk,axis=-1)
# kk = np.repeat(kk,768,axis=-1)
# kk

In [12]:
# bc = SMOTEENN()
# N,t,d = train_emb.shape
# train_emb_ = train_emb.reshape(N,t*d)
# train_fam_ans_ = train_fam_ans.reshape(N,)
# train_emb_ , train_fam_ans_  = bc.fit_resample(train_emb_, train_fam_ans_)

In [13]:
# train_emb = train_emb_.reshape(-1,t,d)
# train_fam_ans = train_fam_ans_.reshape(-1,1)

In [10]:
class_weights = sum(train_fam_ans) / sum(sum(train_fam_ans))
fam_weights={}
for i in range(len(class_weights)):
    fam_weights[i] = 1/class_weights[i]
fam_weights

{0: 4.963642428285524,
 1: 3.4256445672191527,
 2: 11.222473604826547,
 3: 17.163783160322954,
 4: 50.6156462585034,
 5: 63.59401709401709,
 6: 49.60333333333333,
 7: 105.53900709219859,
 8: 56.36742424242424,
 9: 54.70955882352941,
 10: 148.81,
 11: 73.66831683168317,
 12: 73.66831683168317,
 13: 256.5689655172414,
 14: 195.80263157894737,
 15: 25.39419795221843,
 16: 201.09459459459458,
 17: 212.58571428571432,
 18: 153.41237113402062,
 19: 228.93846153846152,
 20: 195.80263157894737,
 21: 144.47572815533982,
 22: 33.7437641723356,
 23: 215.66666666666669,
 24: 165.34444444444443,
 25: 130.53508771929825,
 26: 165.34444444444443,
 27: 338.20454545454544,
 28: 222.1044776119403,
 29: 256.5689655172414,
 30: 64.98253275109171,
 31: 354.3095238095238,
 32: 190.7820512820513,
 33: 163.52747252747253,
 34: 98.54966887417218,
 35: 94.78343949044586,
 36: 362.9512195121951,
 37: 150.31313131313132,
 38: 744.05,
 39: 620.0416666666666,
 40: 391.60526315789474,
 41: 316.6170212765958,
 42: 49

In [11]:
all_fam = sum(train_fam_ans)
for i in range(len(all_fam)):
    fam_weights[i] = all_fam[i]
fam_weights

{0: 2998,
 1: 4344,
 2: 1326,
 3: 867,
 4: 294,
 5: 234,
 6: 300,
 7: 141,
 8: 264,
 9: 272,
 10: 100,
 11: 202,
 12: 202,
 13: 58,
 14: 76,
 15: 586,
 16: 74,
 17: 70,
 18: 97,
 19: 65,
 20: 76,
 21: 103,
 22: 441,
 23: 69,
 24: 90,
 25: 114,
 26: 90,
 27: 44,
 28: 67,
 29: 58,
 30: 229,
 31: 42,
 32: 78,
 33: 91,
 34: 151,
 35: 157,
 36: 41,
 37: 99,
 38: 20,
 39: 24,
 40: 38,
 41: 47,
 42: 30,
 43: 112}

In [12]:
import math
def create_class_weight(labels_dict,mu=0.79):
    total = np.sum(np.array(list(labels_dict.values())))
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight
fam_weights = create_class_weight(fam_weights)
fam_weights

{0: 1.3664174982464374,
 1: 1.0,
 2: 2.1821960061631724,
 3: 2.6070792001284384,
 4: 3.688538409570299,
 5: 3.916797061551279,
 6: 3.6683357022527794,
 7: 4.423358286530812,
 8: 3.7961690737626643,
 9: 3.766316110612983,
 10: 4.766947990920889,
 11: 4.063850479507775,
 12: 4.063850479507775,
 13: 5.311675166362561,
 14: 5.041384836622649,
 15: 2.9987983873319677,
 16: 5.068053083704811,
 17: 5.123622934859622,
 18: 4.7974071984055975,
 19: 5.1977309070133435,
 20: 5.041384836622649,
 21: 4.737389188679344,
 22: 3.2830733014621343,
 23: 5.138011672311721,
 24: 4.872308506578715,
 25: 4.635919728514485,
 26: 4.872308506578715,
 27: 5.587928542990719,
 28: 5.167425557518015,
 29: 5.311675166362561,
 30: 3.938396173354741,
 31: 5.634448558625612,
 32: 5.015409350219389,
 33: 4.8612586703921306,
 34: 4.354838340094056,
 35: 4.315872371560673,
 36: 5.658546110204672,
 37: 4.7769983267743905,
 38: 6.376385903354989,
 39: 6.194064346561035,
 40: 5.734532017182595,
 41: 5.521970575198922,
 42: 

In [17]:
# temp = list(fam_weights.values())
# max_value = np.max(temp)
# for i in range(len(fam_weights)):
#     fam_weights[i] = fam_weights[i]/max_value
# fam_weights

## model hyperparameters

In [14]:
emb_dim = 768 #被除數
num_heads = 1#除數，要整除
max_length = 213 # max sequence length
fam_num = train_fam_ans.shape[1]
vocabulary_size = 26
transformer_depth = 1
transformer_dropout = 0.1

In [15]:
constrain = keras.constraints.MinMaxNorm(min_value=0.0, max_value=0.0, rate=1.0, axis=0)
init = keras.initializers.Ones()
coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth , name='coordinate_embedding')
act_layer = TransformerACT(
            name='adaptive_computation_time')

transformer_block = TransformerBlock(
            name='transformer', num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            # Allow bi-directional attention
            use_masking=False)
add_segment_layer = Add(name='add_segment')

# Model

In [16]:
sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length
                    ,trainable=True,name='api_emb')(sent_ids)



Instructions for updating:
Colocations handled automatically by placer.


In [17]:


next_step_input = sentemb
act_output = sentemb
segment_embeddings = api_emb

for i in range(transformer_depth):
            next_step_input1 = RemoveMask()(next_step_input)
            next_step_input1 = coordinate_embedding_layer(
                next_step_input1, step=i,trainable=False)
            next_step_input = RestoreMask()([next_step_input1,next_step_input])
            next_step_input1 = add_segment_layer(
                [next_step_input, segment_embeddings])
            next_step_input = RemoveMask()(next_step_input1)
            next_step_input = transformer_block(next_step_input)
#             next_step_input = RestoreMask()([next_step_input,next_step_input1])
            next_step_input, act_output = act_layer(next_step_input,trainable=False)
            next_step_input = RestoreMask()([next_step_input,next_step_input1])
            act_output = RestoreMask()([act_output,next_step_input1])

act_layer.finalize()
next_step_input = act_output
# next_step_input = RestoreMask()([next_step_input,act_output])
next_step_input = BatchNormalization()(next_step_input)
att_in = Dense(32,kernel_initializer=keras.initializers.lecun_normal(),activation='selu',
               name='attention_in',trainable=False)(next_step_input)
rep_prediction = (
        Dense(1, name='0_1_prediction', activation='sigmoid')
    (att_in))
# rep = Lambda(lambda x: keras.backend.round(x),name='round')(rep_prediction)
final_emb = Add()([sentemb,api_emb])
mul = multiply([final_emb,rep_prediction],name='mul')
mul = BatchNormalization()(mul)
# dense = Dense(4,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')
multi = MultiHead(GRU(16),layer_num=44,name='multi')(mul)
flat = Flatten(name='Flatten')(multi)
dense = BatchNormalization()(flat)
dense = Dense(176,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')(dense)
dense = Dropout(0.2)(dense)
out = Dense(44,activation='sigmoid', kernel_initializer='lecun_normal',name='family')(dense)

# timesteps,state_h,state_c = LSTM(int(emb_dim/16),return_sequences=True,return_state=True,name='extract'
#                                 , kernel_initializer='lecun_normal',trainable=False)(final_emb) #final_emb
# state = Concatenate()([state_h,state_c])
# fc = Dense(8,kernel_initializer=keras.initializers.lecun_normal(),name='attention_in')(state)
# fc = BatchNormalization()(fc)
# fc = Activation('selu')(fc)
# fc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-2),name='expand')(fc)
# fc = Lambda(lambda x: keras.backend.repeat_elements(x,max_length,axis=-2),name='re')(fc)
# timesteps = Concatenate()([timesteps,fc])
# rep = Dense(1,activation=hard_sigmoid,name='rep',kernel_initializer='lecun_normal')(timesteps)

# fc = Dense(int(emb_dim/2),kernel_initializer=keras.initializers.lecun_normal(),name='attention_in')(state) #multiply_1
# fc = BatchNormalization()(fc)
# fc = Activation('selu')(fc)
# fc = Dense(max_length,activation=hard_sigmoid,name='attention_out')(fc)
# fc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-1),name='expand')(fc)
# fc = Lambda(lambda x: keras.backend.repeat_elements(x,int(emb_dim/4),axis=-1),name='re')(fc)

# fc = keras.backend.repeat_elements(fc,256,axis=-1)
# fc = keras.backend.expand_dims(fc,axis=-1)


# mul = multiply([final_emb,rep],name='mul')
# # mul = BatchNormalization()(mul)
# alls = []
# gru = GRU(int(emb_dim/2), kernel_initializer='lecun_normal',dropout=0.2,recurrent_dropout=0.2,name='fam_input')(mul) #/8
# dense1 = Dense(int(emb_dim/4),kernel_initializer=keras.initializers.lecun_normal(),name='fc1')(gru)
# # gru = GRU(1)
# bn = BatchNormalization()(dense1)
# af = Activation('selu')(bn)
# dp = Dropout(0.2)(af)

# dense1 = Dense(int(emb_dim/8),kernel_initializer=keras.initializers.lecun_normal(),name='fc2')(dp)
# bn = BatchNormalization()(dense1)
# af = Activation('selu')(bn)
# dp = Dropout(0.2)(af)


# dense1 = Dense(int(emb_dim/16),kernel_initializer=keras.initializers.lecun_normal())
# bn = BatchNormalization()
# af = Activation('selu')
# dense = Dense(1,activation='sigmoid')
# for i in range(fam_num):
# #     alls.append(dense(bn(gru(mul))))
#     alls.append(dense(af(bn(dense1(gru)))))
# #     alls.append(gru(mul))
# out = Concatenate(name='family')(alls)

# out = Dense(128,activation='selu',kernel_initializer=keras.initializers.lecun_normal())(out)


# out = Dense(44,activation='sigmoid', kernel_initializer='lecun_normal',name='family')(dp)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
model = Model(inputs=[sent_ids1,sentemb1], outputs=[out]) #out
model.load_weights('./model/att_clf/2ndStage_44fam_0611.h5',by_name=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_emb (InputLayer)           (None, 213, 768)     0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 213, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
sent_ids (InputLayer)           (None, 213)          0                                            
__________________________________________________________________________________________________
remove_mask_1 (RemoveMask)      (None, 213, 768)     0           masking_1[0][0]                  
__________________________________________________________________________________________________
masking_2 

In [23]:
# model = multi_gpu_model(model , gpus=3)

# model.load_weights('./model/LSTM_att/1stStage_44fam_0607.h5')

In [ ]:
'''

sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length,trainable=True,name='api_emb')(sent_ids)

final_emb = Add()([sentemb,api_emb])

timesteps,state_h,state_c = LSTM(int(emb_dim/2),return_sequences=True,return_state=True,name='lstm1')(final_emb) #final_emb
state = Concatenate()([state_h,state_c])
fc = Dense(max_length,activation='sigmoid',bias_constraint=None,kernel_initializer=init,name='attention')(state)
fc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-1),name='RasMMA')(fc)
fc = Lambda(lambda x: keras.backend.repeat_elements(x,int(emb_dim/2),axis=-1))(fc)
# fc = keras.backend.repeat_elements(fc,256,axis=-1)
# fc = keras.backend.expand_dims(fc,axis=-1)
mul = Multiply()([fc,timesteps])
# mul = BatchNormalization()(mul)
alls = []
gru = (GRU(int(emb_dim/4))) #/8
# gru = GRU(1)
bn = BatchNormalization()
dp = Dropout(0.01)

dense = Dense(1,activation='sigmoid')
for i in range(fam_num):
#     alls.append(dense(bn(gru(mul))))
    alls.append(dense(dp(bn(gru(mul)))))
#     alls.append(gru(mul))
out = Concatenate(name='family')(alls)
# out = Dense(44,activation='sigmoid')(out)
model_old = Model(inputs=[sent_ids1,sentemb1], outputs=[out]) #out
model_old = multi_gpu_model(model_old , gpus=3)
model_old.load_weights('./model/LSTM_att/1stStage_44fam_0607.h5')
model_old.summary()

# model = load_model('./model/LSTM_att/1stStage_44fam_0607.h5_all.h5')'''

"\n\nsentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')\nsentemb = Masking(mask_value=0)(sentemb1)\nsent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID\nsent_ids = Masking(mask_value=0)(sent_ids1)\napi_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length,trainable=True,name='api_emb')(sent_ids)\n\nfinal_emb = Add()([sentemb,api_emb])\n\ntimesteps,state_h,state_c = LSTM(int(emb_dim/2),return_sequences=True,return_state=True,name='lstm1')(final_emb) #final_emb\nstate = Concatenate()([state_h,state_c])\nfc = Dense(max_length,activation='sigmoid',bias_constraint=None,kernel_initializer=init,name='attention')(state)\nfc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-1),name='RasMMA')(fc)\nfc = Lambda(lambda x: keras.backend.repeat_elements(x,int(emb_dim/2),axis=-1))(fc)\n# fc = keras.backend.repeat_elements(fc,256,axis=-1)\n# fc = keras.backend.expand_dims(fc,axis=-1)\nmul = Multiply()([fc,timest

In [ ]:
# model_old.layers[-2].save_weights('./model/LSTM_att/test.h5')

In [19]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
from keras.metrics import top_k_categorical_accuracy
def custom_acc1(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred,k=3)
from keras.metrics import binary_accuracy
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

def precision(y_true, y_pred):
    return tf.keras.metrics.Precision(y_true,y_pred)[1]
def recall(y_true, y_pred):
    return tf.keras.metrics.Recall(y_true,y_pred)[1]
# from sklearn.metrics import f1_score
# def f1_sk(y_true,y_pred):
#     score = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
#     return score

# 訓練參數
los = [losses.binary_crossentropy,binary_focal_loss(alpha=.25, gamma=2)] # 1st stage.  f1_loss
#SINGLE
los = [binary_focal_loss(alpha=.25, gamma=2)]

# los = [losses.binary_crossentropy]
#MML
'''los = []
for i in range(fam_num):
    los.append(binary_focal_loss(alpha=.25, gamma=2))
los = [losses.binary_crossentropy] + los'''


metric = {'RasMMA': 'acc','family': f1} # 1st stage. km.f1_score()
#SINGLE
metric = [f1_metric,bin_acc]
# metric = [km.f1_score(),bin_acc,km.binary_f1_score()]
# metric = {'RasMMA': 'acc'}
# metric = [bin_acc]
#MML
'''metrics = []
for i in range(fam_num+1):
    metrics.append('acc')
# metrics = {}
# metrics['RasMMA'] = 'acc'
# for i in range(fam_num):
#     metrics['fam'+str(i)]='acc'
metric = metrics'''


loss_weight = [1,1] #stage1 0.95,0.05  #1st stage # 2nd stage [0.01,0.99]
#SINGLE
loss_weight = [1]
#MML
'''loss_weight = []
for i in range(fam_num):
    loss_weight.append(0.95)
loss_weight = [0.05] + loss_weight'''

learning_rate = 5e-4#2e-4 # 2nd stage: 1e-4 @1st:2e-4 0.002
batch_size = 64 #32 #128

num_epochs = 1000
patien = 50

model_save_path = './model/att_clf/2ndStage_44fam_0611.h5'
tensorboard_log_path = './logs/'+ model_save_path.split('/')[-1].split('.')[0]

In [20]:
# import tensorflow as tf
optimizer = optimizers.Adam(
            lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False) #clipnorm=1. , clipvalue=1.
optimizer = keras.optimizers.Nadam(lr=learning_rate, clipvalue=1.)
# tf.keras.optimizers.Nadam
lr_scheduler1 = callbacks.LearningRateScheduler(
        CosineLRSchedule(lr_high=0.0007, lr_low=1e-8, #learning_rate
                         initial_period=num_epochs),
        verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/3),
                                      min_lr=1e-8,mode='min')

model.compile(
            optimizer,
            loss=los,
            metrics=metric ,loss_weights=loss_weight)#{'word_predictions': masked_perplexity})

In [21]:
# save best, early stopping, 2 models ens weight:(best=0.8,last=0.2)
history = History()
stop_nan = callbacks.TerminateOnNaN()
model_callbacks = [
        callbacks.ModelCheckpoint(
            model_save_path,
            monitor='val_f1_metric',mode='max' ,save_best_only=True, verbose=1,save_weights_only=True),
            EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'),
        lr_scheduler, lr_scheduler1,history,stop_nan
    ]
model_callbacks.append(callbacks.TensorBoard(tensorboard_log_path))

In [22]:
def training_batch(batch_size, X_train1, X_train2 , Y_train1, Y_train2):
    '''
    X_train1 = sent_ids: shape為(N, max_seq_length)
    X_train2 = sentemb: shape為(N,max_seq_length, word_embedding_size)
    Y_train1 = class_prediction: shape為(N, max_seq_length, 1)
    Y_train2 = family_prediction(stage2): shape為(N, fam_num)
    '''
    idx = np.arange(len(X_train1))
    np.random.shuffle(idx)

    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
            train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]
#             yield(train_X2,train_Y2)
#             yield ([train_X1,train_X2],[train_Y1,train_Y2]) #ori
            yield ([train_X1,train_X2],[train_Y2])
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break
            
#     data_size = X_train.shape[0]
#     ep = data_size / batch_size

In [23]:

H = model.fit_generator(
    generator=training_batch(batch_size=batch_size,X_train1=train_emb_api,X_train2=train_emb ,
                                             Y_train1=train_rep_ans,Y_train2=train_fam_ans) #Y_train2
#                     generator=training_batch(batch_size=batch_size,X_train1=valid_emb_api,X_train2=valid_emb ,
#                                              Y_train1=train_rep_ans,Y_train2=train_fam_ans)
                        , steps_per_epoch=int(np.ceil(len(train_emb_api)/batch_size)) ,
                    epochs=num_epochs,callbacks=model_callbacks
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans,valid_fam_ans]) #ori
#                    ,validation_data= (valid_emb, valid_fam_ans) 
                   ,validation_data= ([valid_emb_api,valid_emb], [valid_fam_ans]) #ori
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2) #validY2
                    ,max_queue_size=10  ,class_weight=fam_weights
                    ,workers=10,use_multiprocessing=True   
                   ,shuffle=True,verbose=1)
model.save(model_save_path+"_all.h5")
#1st:train 0_1_prediction=0.14XX

Instructions for updating:
Use tf.cast instead.


/home/leoqaz12/.local/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1000

Epoch 00001: LearningRateScheduler setting learning rate to 0.0007.
175/175 [==============================] - 2350s 13s/step - loss: 35.6001 - f1_metric: 0.6001 - bin_acc: 0.9818 - val_loss: 44.8510 - val_f1_metric: 0.3716 - val_bin_acc: 0.9694

Epoch 00001: val_f1_metric improved from -inf to 0.37162, saving model to ./model/att_clf/2ndStage_44fam_0611.h5
Epoch 2/1000

Epoch 00002: LearningRateScheduler setting learning rate to 0.0006999982728453244.
175/175 [==============================] - 2190s 13s/step - loss: 33.4198 - f1_metric: 0.6160 - bin_acc: 0.9823 - val_loss: 49.9728 - val_f1_metric: 0.3518 - val_bin_acc: 0.9652

Epoch 00002: val_f1_metric did not improve from 0.37162
Epoch 3/1000

Epoch 00003: LearningRateScheduler setting learning rate to 0.0006999930913983436.
175/175 [==============================] - 2039s 12s/step - loss: 34.3756 - f1_metric: 0.6196 - bin_acc: 0.9824 - val_loss: 42.5564 - val_f1_metric: 0.3771 - val_bin_acc: 0.9648

Epoch 00003: val_f

175/175 [==============================] - 2403s 14s/step - loss: 40.3063 - f1_metric: 0.4923 - bin_acc: 0.9786 - val_loss: 39.7818 - val_f1_metric: 0.3640 - val_bin_acc: 0.9662

Epoch 00025: val_f1_metric did not improve from 0.43688
Epoch 26/1000

Epoch 00026: LearningRateScheduler setting learning rate to 0.0006989210822199262.
175/175 [==============================] - 2413s 14s/step - loss: 41.0755 - f1_metric: 0.4963 - bin_acc: 0.9794 - val_loss: 33.9560 - val_f1_metric: 0.3819 - val_bin_acc: 0.9691

Epoch 00026: val_f1_metric did not improve from 0.43688
Epoch 27/1000

Epoch 00027: LearningRateScheduler setting learning rate to 0.0006988330914822703.
175/175 [==============================] - 2404s 14s/step - loss: 37.1321 - f1_metric: 0.5259 - bin_acc: 0.9803 - val_loss: 31.8386 - val_f1_metric: 0.3778 - val_bin_acc: 0.9699

Epoch 00027: val_f1_metric did not improve from 0.43688
Epoch 28/1000

Epoch 00028: LearningRateScheduler setting learning rate to 0.0006987416579521791.
1


Epoch 00049: val_f1_metric did not improve from 0.43731
Epoch 50/1000

Epoch 00050: LearningRateScheduler setting learning rate to 0.0006958612808140027.
175/175 [==============================] - 2119s 12s/step - loss: 41.4934 - f1_metric: 0.4695 - bin_acc: 0.9782 - val_loss: 32.9281 - val_f1_metric: 0.4127 - val_bin_acc: 0.9708

Epoch 00050: val_f1_metric did not improve from 0.43731
Epoch 51/1000

Epoch 00051: LearningRateScheduler setting learning rate to 0.0006956909807665952.
175/175 [==============================] - 2118s 12s/step - loss: 41.3975 - f1_metric: 0.4710 - bin_acc: 0.9780 - val_loss: 34.2004 - val_f1_metric: 0.3929 - val_bin_acc: 0.9697

Epoch 00051: val_f1_metric did not improve from 0.43731
Epoch 52/1000

Epoch 00052: LearningRateScheduler setting learning rate to 0.0006955172689381168.
175/175 [==============================] - 2298s 13s/step - loss: 39.8980 - f1_metric: 0.4632 - bin_acc: 0.9782 - val_loss: 31.5945 - val_f1_metric: 0.4161 - val_bin_acc: 0.9705



175/175 [==============================] - 2303s 13s/step - loss: 41.7962 - f1_metric: 0.4565 - bin_acc: 0.9782 - val_loss: 31.6446 - val_f1_metric: 0.3448 - val_bin_acc: 0.9691

Epoch 00074: val_f1_metric did not improve from 0.43731
Epoch 75/1000

Epoch 00075: LearningRateScheduler setting learning rate to 0.0006905846133511894.
175/175 [==============================] - 2318s 13s/step - loss: 42.7333 - f1_metric: 0.4489 - bin_acc: 0.9785 - val_loss: 27.2792 - val_f1_metric: 0.4390 - val_bin_acc: 0.9718

Epoch 00075: val_f1_metric improved from 0.43731 to 0.43899, saving model to ./model/att_clf/2ndStage_44fam_0611.h5
Epoch 76/1000

Epoch 00076: LearningRateScheduler setting learning rate to 0.0006903296102895848.
175/175 [==============================] - 2303s 13s/step - loss: 41.7435 - f1_metric: 0.4535 - bin_acc: 0.9774 - val_loss: 28.5942 - val_f1_metric: 0.3777 - val_bin_acc: 0.9708

Epoch 00076: val_f1_metric did not improve from 0.43899
Epoch 77/1000

Epoch 00077: LearningRat

# Evaluate

In [ ]:
from keras.models import model_from_json, model_from_yaml
json_string = model.to_json()
yaml_string = model.to_yaml()

In [ ]:
# model.load_weights('./model/LSTM_att/1stStage_44fam_0610.h5')
# model.load_weights('./model/att_clf/1stStage_44fam_0611.h5')
# score = model.evaluate([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2)
print(len(test_emb_api)) #改
ans = model.predict([test_emb_api,test_emb]) #改
y_true = test_fam_ans #改
# ans = model.predict([valid_emb_api,valid_emb])
len(ans)

In [ ]:
test_rep_ans[113]

In [ ]:
ans[113]

In [ ]:
model_.save_weights('./model/LSTM_att/test4.h5')

In [ ]:
model_ = model#.layers[-2]
layer_name = 'lambda_1' #lambda_1 multiply_1  #9~12
intermediate_layer_model = Model(inputs=model_.inputs,
                                 outputs=model_.layers[-3].output)
intermediate_output = intermediate_layer_model.predict([valid_emb_api,valid_emb])
intermediate_output[113]

In [ ]:
sum(sum(intermediate_output))

In [ ]:
intermediate_output[113].shape

In [ ]:
intermediate_output[100].shape

In [ ]:
model_.summary() #multiply_1 

In [ ]:
sum(intermediate_output)

In [ ]:
from sklearn.metrics import f1_score,recall_score,precision_score
# y_true = np.squeeze(test_fam_ans)
# y_true = np.squeeze(valid_fam_ans1)
# y_pred = np.squeeze(predict_fam)
final_ans = []
for sample in ans:
    sample_ans = []
    for value in sample:
        if value < 0.2:
            sample_ans.append(0)
        else:
            sample_ans.append(1)
    final_ans.append(sample_ans)
final_ans = np.array(final_ans)
print(final_ans.shape , sum(final_ans[0]))

In [ ]:
y_true = test_fam_ans
print(y_true.shape , final_ans.shape)
recall = recall_score(y_true=y_true, y_pred=final_ans, average='weighted')
precision = precision_score(y_true=y_true, y_pred=final_ans, average='weighted')
f1 = f1_score(y_true=y_true, y_pred=final_ans, average='weighted')
recall ,precision, f1